## Classifying Meme Images into Hateful or Not-Hateful 
### Model 2: Training MobileNetV2

### Step 1: Importing the required libraries

In [17]:
import tensorflow as tf
import os
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

### Step 2: Setting up Paths

In [18]:
source_folder = "img"
train_folder = "train_data"
validation_folder = "validation_data"

train_dir = os.path.join(source_folder, train_folder)
validation_dir = os.path.join(source_folder, validation_folder)

# Various Directory Paths
train_hateful_memes_dir = os.path.join(train_dir, 'hateful_memes')
train_not_hateful_memes_dir = os.path.join(train_dir, 'not_hateful_memes')
validation_hateful_memes_dir = os.path.join(validation_dir, 'hateful_memes')
validation_not_hateful_memes_dir = os.path.join(validation_dir, 'not_hateful_memes')

### Step 3: Checking Count of Images in Dataset

In [19]:
print('total training hateful images:', len(os.listdir(train_hateful_memes_dir)))
print('total training not hateful images:', len(os.listdir(train_not_hateful_memes_dir)))
print('total validation hateful images:', len(os.listdir(validation_hateful_memes_dir)))
print('total validation not hateful images:', len(os.listdir(validation_not_hateful_memes_dir)))

total training hateful images: 2400
total training not hateful images: 2400
total validation hateful images: 600
total validation not hateful images: 600


### Step 4: Performing Data-Augmentation using ImageDataGenerator

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data-augmentation parameters to ImageDataGenerator for Training set

train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale=1/255)

### Step 5: Setting up Training and Validation Generators

In [21]:
# Flow training images in batches of 1 using train_datagen generator

train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        classes = ['hateful_memes', 'not_hateful_memes'], #The Classification
        target_size=(224, 224),  # All images will be resized to 224x224
        batch_size=1,
        class_mode='binary') # Use binary labels


# Flow validation images in batches of 1 using valid_datagen generator

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,  # This is the source directory for validating images
        classes = ['hateful_memes', 'not_hateful_memes'], #The Classification
        target_size=(224, 224),  # All images will be resized to 224x224
        batch_size=1,
        class_mode='binary', # Use binary labels
        shuffle=False)

Found 4800 images belonging to 2 classes.


Found 1200 images belonging to 2 classes.


### Step 6: Setting up the MobileNetV2 Model for Training

In [22]:
model = Sequential()
model.add(MobileNetV2(include_top = False, weights="imagenet", input_shape=(224, 224, 3)))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Dense(1, activation = 'sigmoid'))
model.layers[0].trainable = False

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_2  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             (None, 1)                 1281      
                                                                 
Total params: 2259265 (8.62 MB)
Trainable params: 1281 (5.00 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


### Step 7: Choosing the Training Parameters

In [23]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import optimizers

model.compile(optimizer=RMSprop(lr=0.001), loss = 'binary_crossentropy', metrics = 'accuracy')

### Step 8: Start the Training of the model

In [24]:
history = model.fit(
      train_generator,
      steps_per_epoch=4800,  
      epochs=10,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=1200)

Epoch 1/10
4800/4800 [==============================] - 107s 22ms/step - loss: 0.8196 - accuracy: 0.5337 - val_loss: 0.8297 - val_accuracy: 0.5400
Epoch 2/10
4800/4800 [==============================] - 104s 22ms/step - loss: 0.7994 - accuracy: 0.5506 - val_loss: 0.7519 - val_accuracy: 0.5733
Epoch 3/10
4800/4800 [==============================] - 105s 22ms/step - loss: 0.7837 - accuracy: 0.5723 - val_loss: 0.7601 - val_accuracy: 0.5575
Epoch 4/10
4800/4800 [==============================] - 110s 23ms/step - loss: 0.7828 - accuracy: 0.5758 - val_loss: 0.8938 - val_accuracy: 0.5317
Epoch 5/10
4800/4800 [==============================] - 104s 22ms/step - loss: 0.7778 - accuracy: 0.5792 - val_loss: 0.7913 - val_accuracy: 0.5650
Epoch 6/10
4800/4800 [==============================] - 105s 22ms/step - loss: 0.7843 - accuracy: 0.5815 - val_loss: 0.8032 - val_accuracy: 0.5717
Epoch 7/10
4800/4800 [==============================] - 108s 22ms/step - loss: 0.7823 - accuracy: 0.5919 - val_loss: 0

### Step 9: Start Validation of the model



In [25]:
val_loss, val_acc = model.evaluate(validation_generator)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_acc)

1200/1200 [==============================] - 21s 18ms/step - loss: 0.8392 - accuracy: 0.5642
Validation Loss: 0.8392002582550049
Validation Accuracy: 0.5641666650772095


### Step 10: Saving the Model

In [27]:
model.save('Classification_MobileNet.keras')

### Step 11: Testing the Model for Classification

In [ ]:
local_image_directory = "" #Insert the Folder Path containing the images
from keras.preprocessing import image
import matplotlib.pyplot as plt

# List all files in the directory
image_files = [f for f in os.listdir(local_image_directory) if os.path.isfile(os.path.join(local_image_directory, f))]

for fn in image_files:
    # Predicting images
    path = os.path.join(local_image_directory, fn)  # Updated to use os.path.join for the file path
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = model.predict(images, batch_size=1)
    if classes[0] < 0.5:
        print(fn + " is a Hateful Meme")
    else:
        print(fn + " is a Not Hateful Meme")